In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/instance_version_enhanced-20250720T221706Z-1-001.zip -d /content


Streaming output truncated to the last 5000 lines.
  inflating: /content/instance_version_enhanced/train/vid_000122_frame0000037.jpg  
  inflating: /content/instance_version_enhanced/train/vid_000081_frame0000046.jpg  
  inflating: /content/instance_version_enhanced/train/vid_000269_frame0000058.jpg  
  inflating: /content/instance_version_enhanced/train/vid_000158_frame0000118.jpg  
  inflating: /content/instance_version_enhanced/train/vid_000170_frame0000012.jpg  
  inflating: /content/instance_version_enhanced/val/vid_000277_frame0000021.jpg  
  inflating: /content/instance_version_enhanced/train/vid_000101_frame0000003.jpg  
  inflating: /content/instance_version_enhanced/train/vid_000144_frame0000018.jpg  
  inflating: /content/instance_version_enhanced/train/vid_000551_frame0000124.jpg  
  inflating: /content/instance_version_enhanced/train/vid_000072_frame0000028.jpg  
  inflating: /content/instance_version_enhanced/train/vid_000365_frame0000003.jpg  
  inflating: /content/insta

In [ ]:
%cd /content/instance_version_enhanced/

/content/instance_version_enhanced


In [ ]:
import json

with open("instances_train_trashcan.json") as f:
    train_data = json.load(f)

with open("instances_val_trashcan.json") as f:
    val_data = json.load(f)

print("Top-level keys:", train_data.keys())
print("Top-level keys:", val_data.keys())

Top-level keys: dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])
Top-level keys: dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])


In [ ]:
import os
import json
from tqdm import tqdm
from PIL import Image
from pycocotools.coco import COCO

def convert_coco_to_yolo(coco_json_path, images_dir, output_labels_dir):
    os.makedirs(output_labels_dir, exist_ok=True)

    coco = COCO(coco_json_path)
    img_id_to_filename = {img['id']: img['file_name'] for img in coco.dataset['images']}
    img_id_to_size = {img['id']: (img['width'], img['height']) for img in coco.dataset['images']}

    for ann in tqdm(coco.dataset['annotations']):
        img_id = ann['image_id']
        category_id = ann['category_id'] - 1
        bbox = ann['bbox']

        x, y, w, h = bbox
        img_width, img_height = img_id_to_size[img_id]

        x_center = (x + w / 2) / img_width
        y_center = (y + h / 2) / img_height
        w /= img_width
        h /= img_height

        # Build annotation line
        annotation_line = f"{category_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n"

        # Write to label file
        img_filename = img_id_to_filename[img_id]
        label_filename = os.path.splitext(img_filename)[0] + '.txt'
        label_path = os.path.join(output_labels_dir, label_filename)

        with open(label_path, 'a') as f:
            f.write(annotation_line)

    print("✅ Conversion complete!")

# Example usage
convert_coco_to_yolo(
    coco_json_path="instances_train_trashcan.json",
    images_dir="/content/instance_version_enhanced/train",
    output_labels_dir="/content/yolo_dataset/labels/train"
)

convert_coco_to_yolo(
    coco_json_path="instances_val_trashcan.json",
    images_dir="/content/instance_version_enhanced/val",
    output_labels_dir="/content/yolo_dataset/labels/val"
)


loading annotations into memory...
Done (t=0.29s)
creating index...
index created!


100%|██████████| 9540/9540 [00:00<00:00, 25768.60it/s]


✅ Conversion complete!
loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


100%|██████████| 2588/2588 [00:00<00:00, 29865.26it/s]

✅ Conversion complete!


In [ ]:
import shutil
import os

# Create image dirs
os.makedirs("/content/yolo_dataset/images/train", exist_ok=True)
os.makedirs("/content/yolo_dataset/images/val", exist_ok=True)

# Paths to your original images
train_imgs = "/content/instance_version_enhanced/train"
val_imgs = "/content/instance_version_enhanced/val"

# Copy images
for fname in os.listdir(train_imgs):
    if fname.endswith(".jpg"):
        shutil.copy(os.path.join(train_imgs, fname), "/content/yolo_dataset/images/train")

for fname in os.listdir(val_imgs):
    if fname.endswith(".jpg"):
        shutil.copy(os.path.join(val_imgs, fname), "/content/yolo_dataset/images/val")

In [ ]:
import os
import shutil
import random

# Define paths
img_train_path = "/content/yolo_dataset/images/train"
lbl_train_path = "/content/yolo_dataset/labels/train"
img_test_path = "/content/yolo_dataset/images/test"
lbl_test_path = "/content/yolo_dataset/labels/test"

# Create test folders
os.makedirs(img_test_path, exist_ok=True)
os.makedirs(lbl_test_path, exist_ok=True)

# Get list of all training images
train_images = [f for f in os.listdir(img_train_path) if f.endswith(".jpg")]

# Split 10% for test
test_size = int(0.1 * len(train_images))
test_images = random.sample(train_images, test_size)

# Move images and corresponding labels to test folders
for img in test_images:
    label = img.replace(".jpg", ".txt")

    # Move image
    src_img = os.path.join(img_train_path, img)
    dst_img = os.path.join(img_test_path, img)
    shutil.move(src_img, dst_img)

    # Move label (if it exists)
    src_lbl = os.path.join(lbl_train_path, label)
    dst_lbl = os.path.join(lbl_test_path, label)
    if os.path.exists(src_lbl):
        shutil.move(src_lbl, dst_lbl)


In [ ]:

print(f"Number of images in /content/yolo_dataset/images/train: {len(os.listdir('/content/yolo_dataset/images/train'))}")
print(f"Number of images in /content/yolo_dataset/images/val: {len(os.listdir('/content/yolo_dataset/images/val'))}")
print(f"Number of images in /content/yolo_dataset/images/test: {len(os.listdir('/content/yolo_dataset/images/test'))}")

Number of images in /content/yolo_dataset/images/train: 5459
Number of images in /content/yolo_dataset/images/val: 1147
Number of images in /content/yolo_dataset/images/test: 606


In [ ]:
import os
from glob import glob

def check_labels(label_dir):
    bad_files = []
    for label_file in glob(f"{label_dir}/**/*.txt", recursive=True):
        with open(label_file, 'r') as f:
            lines = f.readlines()
            for line in lines:
                parts = line.strip().split()
                if len(parts) != 5:
                    bad_files.append(label_file)
                    break
                coords = list(map(float, parts[1:]))
                if any(c < 0 or c > 1 for c in coords):
                    bad_files.append(label_file)
                    break
    return list(set(bad_files))

bad_labels = check_labels("/content/yolo_dataset/labels/train")
print(f"Found {len(bad_labels)} corrupted label files:")

Found 0 corrupted label files:


In [ ]:
from PIL import Image

def renormalize_label_file(label_path, images_dir):
    img_name = os.path.basename(label_path).replace(".txt", ".jpg")
    img_path = os.path.join(images_dir, img_name)

    if not os.path.exists(img_path):
        print(f"⚠️ Image not found for: {label_path}")
        return

    with Image.open(img_path) as img:
        img_width, img_height = img.size

    fixed_lines = []
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue

            cls, x, y, w, h = parts
            try:
                x, y, w, h = map(float, [x, y, w, h])
            except:
                continue

            # Denormalize and re-normalize (with clamping)
            x_pixel = x * img_width
            y_pixel = y * img_height
            w_pixel = w * img_width
            h_pixel = h * img_height

            x_new = min(max(x_pixel / img_width, 0), 1)
            y_new = min(max(y_pixel / img_height, 0), 1)
            w_new = min(max(w_pixel / img_width, 0), 1)
            h_new = min(max(h_pixel / img_height, 0), 1)

            fixed_lines.append(f"{cls} {x_new:.6f} {y_new:.6f} {w_new:.6f} {h_new:.6f}\n")

    # Save fixed label file
    with open(label_path, 'w') as f:
        f.writelines(fixed_lines)


for label_path in bad_labels:
    renormalize_label_file(label_path, "/content/yolo_dataset/images/train")

In [ ]:
import os
from glob import glob

def check_labels(label_dir):
    bad_files = []
    for label_file in glob(f"{label_dir}/**/*.txt", recursive=True):
        with open(label_file, 'r') as f:
            lines = f.readlines()
            for line in lines:
                parts = line.strip().split()
                if len(parts) != 5:
                    bad_files.append(label_file)
                    break
                coords = list(map(float, parts[1:]))
                if any(c < 0 or c > 1 for c in coords):
                    bad_files.append(label_file)
                    break
    return list(set(bad_files))

bad_labels = check_labels("/content/yolo_dataset/labels/val")
print(f"Found {len(bad_labels)} corrupted label files:")

Found 0 corrupted label files:


In [ ]:
from PIL import Image

def renormalize_label_file(label_path, images_dir):
    img_name = os.path.basename(label_path).replace(".txt", ".jpg")
    img_path = os.path.join(images_dir, img_name)

    if not os.path.exists(img_path):
        print(f"⚠️ Image not found for: {label_path}")
        return

    with Image.open(img_path) as img:
        img_width, img_height = img.size

    fixed_lines = []
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue

            cls, x, y, w, h = parts
            try:
                x, y, w, h = map(float, [x, y, w, h])
            except:
                continue

            # Denormalize and re-normalize (with clamping)
            x_pixel = x * img_width
            y_pixel = y * img_height
            w_pixel = w * img_width
            h_pixel = h * img_height

            x_new = min(max(x_pixel / img_width, 0), 1)
            y_new = min(max(y_pixel / img_height, 0), 1)
            w_new = min(max(w_pixel / img_width, 0), 1)
            h_new = min(max(h_pixel / img_height, 0), 1)

            fixed_lines.append(f"{cls} {x_new:.6f} {y_new:.6f} {w_new:.6f} {h_new:.6f}\n")

    # Save fixed label file
    with open(label_path, 'w') as f:
        f.writelines(fixed_lines)


for label_path in bad_labels:
    renormalize_label_file(label_path, "/content/yolo_dataset/images/val")

In [ ]:
import os
from glob import glob

def check_labels(label_dir):
    bad_files = []
    for label_file in glob(f"{label_dir}/**/*.txt", recursive=True):
        with open(label_file, 'r') as f:
            lines = f.readlines()
            for line in lines:
                parts = line.strip().split()
                if len(parts) != 5:
                    bad_files.append(label_file)
                    break
                coords = list(map(float, parts[1:]))
                if any(c < 0 or c > 1 for c in coords):
                    bad_files.append(label_file)
                    break
    return list(set(bad_files))

bad_labels = check_labels("/content/yolo_dataset/labels/test")
print(f"Found {len(bad_labels)} corrupted label files:")

Found 0 corrupted label files:


In [ ]:
from PIL import Image

def renormalize_label_file(label_path, images_dir):
    img_name = os.path.basename(label_path).replace(".txt", ".jpg")
    img_path = os.path.join(images_dir, img_name)

    if not os.path.exists(img_path):
        print(f"⚠️ Image not found for: {label_path}")
        return

    with Image.open(img_path) as img:
        img_width, img_height = img.size

    fixed_lines = []
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue

            cls, x, y, w, h = parts
            try:
                x, y, w, h = map(float, [x, y, w, h])
            except:
                continue

            # Denormalize and re-normalize (with clamping)
            x_pixel = x * img_width
            y_pixel = y * img_height
            w_pixel = w * img_width
            h_pixel = h * img_height

            x_new = min(max(x_pixel / img_width, 0), 1)
            y_new = min(max(y_pixel / img_height, 0), 1)
            w_new = min(max(w_pixel / img_width, 0), 1)
            h_new = min(max(h_pixel / img_height, 0), 1)

            fixed_lines.append(f"{cls} {x_new:.6f} {y_new:.6f} {w_new:.6f} {h_new:.6f}\n")

    # Save fixed label file
    with open(label_path, 'w') as f:
        f.writelines(fixed_lines)


for label_path in bad_labels:
    renormalize_label_file(label_path, "/content/yolo_dataset/images/test")

In [ ]:
yaml_content = """
train: /content/yolo_dataset/images/train
val: /content/yolo_dataset/images/val
test: /content/yolo_dataset/images/test

nc: 22
names:
  - rov
  - plant
  - animal_fish
  - animal_starfish
  - animal_shells
  - animal_crab
  - animal_eel
  - animal_etc
  - trash_clothing
  - trash_pipe
  - trash_bottle
  - trash_bag
  - trash_snack_wrapper
  - trash_can
  - trash_cup
  - trash_container
  - trash_unknown_instance
  - trash_branch
  - trash_wreckage
  - trash_tarp
  - trash_rope
  - trash_net
"""

with open("/content/yolo_dataset/data.yaml", "w") as f:
    f.write(yaml_content)


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
# Go to the /content directory
%cd /content

# Clone the source code from GitHub
!git clone https://github.com/ultralytics/ultralytics.git

/content
Cloning into 'ultralytics'...
remote: Enumerating objects: 62060, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 62060 (delta 0), reused 0 (delta 0), pack-reused 62059 (from 1)
Receiving objects: 100% (62060/62060), 33.43 MiB | 25.97 MiB/s, done.
Resolving deltas: 100% (46320/46320), done.


In [ ]:
%cd /content/ultralytics

/content/ultralytics


In [ ]:
!pip install -e .

Obtaining file:///content/ultralytics
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ultralytics (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.3.168-0.editable-py3-none-any.whl size=23146 sha256=4fae1d4d573a3328b06ff9d429e8033a57fab6d0ebbbdc0bc19a608912f2fe4e
  Stored in directory: /tmp/pip-ephem-wheel-cache-5e1rycot/wheels/ea/71/6b/a9012dfb148489fd8125c2310e565414c996b3c7721defe799
Successfully built ultralytics
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.3.168
    Uninstalling ultralytics-8.3.168:
      Successfully uninstalled ultralytics-8.3.168


In [ ]:
# Create a new cell and run this code.
# It defines the CBAM module and writes it to ultralytics/nn/custom_modules.py
custom_module_code = """
import torch
import torch.nn as nn
import torch.nn.functional as F
from timm.models.layers import Mlp, DropPath, to_2tuple, trunc_normal_
from ultralytics.nn.modules import Conv

class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        print(f"DEBUG: Initializing ChannelAttention with in_planes={in_planes}, ratio={ratio}")
        if in_planes < ratio:
          print(f"!!! CRITICAL WARNING: in_planes ({in_planes}) is less than ratio ({ratio}")
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc = nn.Sequential(nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
                                nn.ReLU(),
                                nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False))
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size // 2, bias=False)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x_cat = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x_cat)
        return self.sigmoid(x)

class CBAM(nn.Module):
    def __init__(self, c1, ratio=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.channel_attn = ChannelAttention(c1, ratio)
        self.spatial_attn = SpatialAttention(kernel_size)
    def forward(self, x):
        x = x * self.channel_attn(x)
        x = x * self.spatial_attn(x)
        return x



def window_partition(x, window_size):

    B, H, W, C = x.shape
    x = x.view(B, H // window_size, window_size, W // window_size, window_size, C)
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(-1, window_size, window_size, C)
    return windows

def window_reverse(windows, window_size, H, W):

    B = int(windows.shape[0] / (H * W / window_size / window_size))
    x = windows.view(B, H // window_size, W // window_size, window_size, window_size, -1)
    x = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(B, H, W, -1)
    return x

class WindowAttention(nn.Module):
    # This is a simplified version for clarity, adapted from timm
    def __init__(self, dim, window_size, num_heads, qkv_bias=True, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.dim = dim
        self.window_size = window_size
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask=None):
        B_, N, C = x.shape
        qkv = self.qkv(x).reshape(B_, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        q = q * self.scale
        attn = (q @ k.transpose(-2, -1))

        if mask is not None:
            nW = mask.shape[0]
            attn = attn.view(B_ // nW, nW, self.num_heads, N, N) + mask.unsqueeze(1).unsqueeze(0)
            attn = attn.view(-1, self.num_heads, N, N)
            attn = self.softmax(attn)
        else:
            attn = self.softmax(attn)

        attn = self.attn_drop(attn)
        x = (attn @ v).transpose(1, 2).reshape(B_, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

class SwinTransformerBlock(nn.Module):
    # This is a simplified version for clarity, adapted from timm
    def __init__(self, dim, num_heads, window_size=7, shift_size=0, mlp_ratio=4., qkv_bias=True, drop=0., attn_drop=0., drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio

        self.norm1 = norm_layer(dim)
        self.attn = WindowAttention(dim, window_size=to_2tuple(self.window_size), num_heads=num_heads, qkv_bias=qkv_bias, attn_drop=attn_drop, proj_drop=drop)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

    def forward(self, x, H, W):
        B, L, C = x.shape
        shortcut = x
        x = self.norm1(x)
        x = x.view(B, H, W, C)

        # Pad feature map to a multiple of window size
        pad_l = pad_t = 0
        pad_r = (self.window_size - W % self.window_size) % self.window_size
        pad_b = (self.window_size - H % self.window_size) % self.window_size
        x = F.pad(x, (0, 0, pad_l, pad_r, pad_t, pad_b))
        _, Hp, Wp, _ = x.shape

        # Cyclic shift
        if self.shift_size > 0:
            shifted_x = torch.roll(x, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2))
            attn_mask = None
        else:
            shifted_x = x
            attn_mask = None

        # Partition windows
        x_windows = window_partition(shifted_x, self.window_size)
        x_windows = x_windows.view(-1, self.window_size * self.window_size, C)

        # W-MSA/SW-MSA
        attn_windows = self.attn(x_windows, mask=attn_mask)

        # Merge windows
        attn_windows = attn_windows.view(-1, self.window_size, self.window_size, C)
        shifted_x = window_reverse(attn_windows, self.window_size, Hp, Wp)

        # Reverse cyclic shift
        if self.shift_size > 0:
            x = torch.roll(shifted_x, shifts=(self.shift_size, self.shift_size), dims=(1, 2))
        else:
            x = shifted_x

        if pad_r > 0 or pad_b > 0:
            x = x[:, :H, :W, :].contiguous()

        x = x.view(B, H * W, C)
        x = shortcut + self.drop_path(self.mlp(self.norm2(x)))
        return x

# THE IMPORTANT WRAPPER MODULE
class SwinLayer(nn.Module):
    def __init__(self, c1, c2, num_heads=8, num_blocks=1, window_size=7):
        super().__init__()
        self.conv = None
        if c1 != c2:
            self.conv = Conv(c1, c2)

        self.blocks = nn.ModuleList([
            SwinTransformerBlock(dim=c2, num_heads=num_heads, window_size=window_size,
                                 shift_size=0 if (i % 2 == 0) else window_size // 2)
            for i in range(num_blocks)
        ])

    def forward(self, x):
        if self.conv:
            x = self.conv(x)

        # Permute from (B, C, H, W) to (B, H, W, C) for Swin Block
        b, c, h, w = x.shape
        x_permuted = x.permute(0, 2, 3, 1)
        x_flattened = x_permuted.view(b, h * w, c)

        # Pass through Swin blocks
        for block in self.blocks:
            x_flattened = block(x_flattened, h, w)

        # Reshape and permute back to (B, C, H, W)
        x_reshaped = x_flattened.view(b, h, w, c)
        x_final = x_reshaped.permute(0, 3, 1, 2)
        return x_final
"""

with open("ultralytics/nn/custom_modules.py", "w") as f:
    f.write(custom_module_code)

print("custom_modules.py created successfully with definition.")

custom_modules.py created successfully with definition.


In [ ]:
from ultralytics.nn.custom_modules import SwinLayer, CBAM

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [ ]:
hybrid_model_yaml = """
# yolov8-Swin-cbam.yaml

nc: 22
depth_multiple: 0.67  # model depth multiple
width_multiple: 0.75  # layer channel multiple

# YOLOv8 backbone
backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]]   # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]]  # 1-P2/4
  - [-1, 3, C2f, [128, True]]   # 2
  - [-1, 1, Conv, [256, 3, 2]]  # 3-P3/8
  - [-1, 6, C2f, [256, True]]   # 4
  - [-1, 1, Conv, [512, 3, 2]]  # 5-P4/16
  - [-1, 1, SwinLayer, [512, 16]] # 6, c1=512, c2=512, heads=16
  - [-1, 1, Conv, [1024, 3, 2]] # 7-P5/32
  - [-1, 3, C2f, [1024, True]]  # 8
  - [-1, 1, SPPF, [1024, 5]]    # 9

# YOLOv8 head
head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']] # 10
  - [[-1, 6], 1, Concat, [1]]   # 11 cat backbone P4
  - [-1, 3, C2f, [512, False]]  # 12
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']] # 13
  - [[-1, 4], 1, Concat, [1]]   # 14 cat backbone P3
  - [-1, 3, C2f, [256, False]]  # 15 (P3/8-small)
  - [-1, 1, CBAM, [16, 7]]      # 16, c1=256, ratio=16, kernel=7

  - [-1, 1, Conv, [256, 3, 2]]  # 17
  - [[-1, 12], 1, Concat, [1]]  # 18 cat head P4
  - [-1, 3, C2f, [512, False]]  # 19 (P4/16-medium)
  - [-1, 1, CBAM, [16, 7]]      # 20, c1=512, ratio=16, kernel=7

  - [-1, 1, Conv, [512, 3, 2]]  # 21
  - [[-1, 9], 1, Concat, [1]]   # 22 cat head P5
  - [-1, 3, C2f, [1024, False]] # 23 (P5/32-large)
  - [-1, 1, CBAM, [16, 7]]      # 24, c1=1024, ratio=16, kernel=7

  # Detect head
  - [[16, 20, 24], 1, Detect, [nc]] # 25

"""

# Write the model YAML to a file
with open("yolov8-Swin-cbam.yaml", "w") as f:
    f.write(hybrid_model_yaml)

print("Model file created successfully.")

Model file created successfully.


In [ ]:
!yolo train \
    model='yolov8-Swin-cbam.yaml' \
    pretrained='yolov8m.pt' \
    data='/content/yolo_dataset/data.yaml' \
    epochs=120 \
    imgsz=640 \
    batch=16 \
    name=yolov8_swin_cbam_Enhanced \
    project=/content/drive/MyDrive/Yolov8-Enhanced \
    hsv_h=0.025 \
    hsv_s=0.9 \
    hsv_v=0.5 \
    degrees=10.0 \
    translate=0.1 \
    scale=0.6 \
    shear=5.0 \
    perspective=0.0005 \
    copy_paste=0.15

/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
DEBUG: Initializing ChannelAttention with in_planes=192, ratio=16
DEBUG: Initializing ChannelAttention with in_planes=384, ratio=16
DEBUG: Initializing ChannelAttention with in_planes=768, ratio=16
100% 49.7M/49.7M [00:00<00:00, 355MB/s]
Transferred 200/442 items from pretrained weights
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.15, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_dataset/data.yaml, degrees=10.0, deterministic=True, device=None, dfl=1.5, dnn=F

In [ ]:
!yolo val \
    model=/content/drive/MyDrive/Yolov8-Enhanced/yolov8_swin_cbam_Enhanced2/weights/best.pt \
    data=/content/yolo_dataset/data.yaml \
    split=test \
    imgsz=640 \
    conf=0.001 \
    iou=0.6 \
    half=False

/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLOv8-Swin-cbam summary (fused): 121 layers, 35,080,280 parameters, 0 gradients, 124.6 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 665.1±279.8 MB/s, size: 14.8 KB)
val: Scanning /content/yolo_dataset/labels/test... 596 images, 10 backgrounds, 0 corrupt: 100% 606/606 [00:00<00:00, 1623.98it/s]
val: New cache created: /content/yolo_dataset/labels/test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 38/38 [00:06<00:00,  5.80it/s]
                   all        606        975      0.893      0.915      0.945      0.731
                   rov        222        264      0.96

In [ ]:
!yolo task=detect \
  mode=predict \
  model=/content/drive/MyDrive/Yolov8-Enhanced/yolov8_swin_cbam_Enhanced2/weights/best.pt \
  source=/content/yolo_dataset/images/test/vid_000104_frame0000016.jpg \
  imgsz=640

/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLOv8-Swin-cbam summary (fused): 121 layers, 35,080,280 parameters, 0 gradients, 124.6 GFLOPs

image 1/1 /content/yolo_dataset/images/test/vid_000104_frame0000016.jpg: 384x640 2 rovs, 1 animal_eel, 69.7ms
Speed: 3.0ms preprocess, 69.7ms inference, 133.3ms postprocess per image at shape (1, 3, 384, 640)
Results saved to /content/ultralytics/runs/detect/predict4
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
!zip -r /content/test_images.zip /content/yolo_dataset/images/test


  adding: content/yolo_dataset/images/test/ (stored 0%)
  adding: content/yolo_dataset/images/test/vid_000251_frame0000005.jpg (deflated 4%)
  adding: content/yolo_dataset/images/test/vid_000532_frame0000053.jpg (deflated 1%)
  adding: content/yolo_dataset/images/test/vid_000075_frame0000190.jpg (deflated 2%)
  adding: content/yolo_dataset/images/test/vid_000316_frame0000034.jpg (deflated 1%)
  adding: content/yolo_dataset/images/test/vid_000158_frame0000074.jpg (deflated 2%)
  adding: content/yolo_dataset/images/test/vid_000242_frame0000008.jpg (deflated 2%)
  adding: content/yolo_dataset/images/test/vid_000363_frame0000012.jpg (deflated 2%)
  adding: content/yolo_dataset/images/test/vid_000149_frame0000002.jpg (deflated 2%)
  adding: content/yolo_dataset/images/test/vid_000120_frame0000004.jpg (deflated 2%)
  adding: content/yolo_dataset/images/test/vid_000332_frame0000001.jpg (deflated 1%)
  adding: content/yolo_dataset/images/test/vid_000344_frame0000029.jpg (deflated 1%)
  adding: